In [1]:
import tensorflow as tf
import numpy as np
import keras.backend as K
import math

Using TensorFlow backend.


In [2]:
# image = (batch, width, height, channel) (assumed)
# GLCM_subarray = (batch, width', height', channel, l_for_GLCM, l_for_GLCM, l_for_Lagrange_polynomial, l_for_Lagrange_polynomial)

In [3]:
# an instance of input
inputs = np.random.randint(0, 3, size=(1, 3, 3, 1))
print("inputs")
inputs_for_test = np.reshape(inputs, (3, 3))
print(inputs_for_test)
image = tf.constant(inputs, dtype=tf.float32)

#some parameters of GLCM
level = 3
distance = 2
angle = math.pi
symmetric = True
normed = True

# Most of time, we will only know batch_size while executing time, 
# however, to do tf.reshape and tf.tile, we have to fix it first.
batch_size = inputs.shape[0]

#make image to be regularized to the fixed levels.
image = tf.cast(image/(3/level), dtype=tf.int32)
#image = tf.cast(image, dtype=tf.float32)

#print("Check some value: ")
#print(K.eval(image[0][0:5][0:5]))

print("image_shape = ", image.shape)

inputs
[[2 2 2]
 [0 0 2]
 [2 1 2]]
image_shape =  (1, 3, 3, 1)


In [4]:
#Check answer with skimage
from skimage.feature import greycomatrix

glcm = greycomatrix(inputs_for_test, distances=[2], angles=[math.pi], levels=3, symmetric=True, normed=True)

#To make it better to read, we reshaped it.
print(np.reshape(glcm, (glcm.shape[0], glcm.shape[1])))


[[ 0.          0.          0.16666667]
 [ 0.          0.          0.        ]
 [ 0.16666667  0.          0.66666667]]


In [5]:
#get some parameters from image
batch = batch_size
rows = image.shape[1].value
cols = image.shape[2].value
channel = image.shape[3].value


#calculate the subarray size of image 
row = int(round(np.sin(angle))) * distance
col = int(round(np.cos(angle))) * distance

#We have two subarrays with same shape to express the two distinct entries
#We wil use Matrix operations instead of for-loop and if, for the ban on them from tensorflow. 
if col > 0:
    subarray_1 = image[:, :rows-row, :cols-col, :]
    subarray_2 = image[:, row:, col:, :]
else:
    subarray_1 = image[:, :rows-row, -col:, :]
    subarray_2 = image[:, row:, :cols+col, :]

print("subarray_1.shape: ", subarray_1.shape)
print("subarray_2.shape: ", subarray_2.shape)

# get someparameters from subarray, too.
sub_row = subarray_1.shape[1].value
sub_column = subarray_1.shape[2].value

print("Subarray_1: ")
print(K.eval(tf.reshape(subarray_1, (subarray_1.shape[1].value, subarray_1.shape[2].value))))
print("Subarray_2: ")
print(K.eval(tf.reshape(subarray_2, (subarray_2.shape[1].value, subarray_2.shape[2].value))))


subarray_1.shape:  (1, 3, 1, 1)
subarray_2.shape:  (1, 3, 1, 1)
Subarray_1: 
[[2]
 [2]
 [2]]
Subarray_2: 
[[2]
 [0]
 [2]]


In [6]:
# To implement Lagrange polynomial, and the calculation on every entry of GLCM, 
# subarrays have to be reshaped and tiled the as
# (batch, width', height', channel, level_for_GLCM, level_for_GLCM, level-1_for_Lagrange_polynomial)
# two level_for_GLCM is because of a_{i, j}, 
# and level_for_Lagrange_polynomial is for (0, ... , level-1), but only pick (level-1) distinct values in Lagrange polynomials.

# For tensorflow can only tile for dim<5, we will do like: "tile->reshape" most of time.

reshaped_subarray_1 = tf.reshape(subarray_1, shape=(batch, sub_row, sub_column, channel, 1))
reshaped_subarray_2 = tf.reshape(subarray_2, shape=(batch, sub_row, sub_column, channel, 1))

print("reshaped_subarray_1: ", reshaped_subarray_1)
print("reshaped_subarray_2: ", reshaped_subarray_2)

tiled_subarray_1 = tf.tile(reshaped_subarray_1, [1, 1, 1, 1, level*level*(level-1)])
tiled_subarray_2 = tf.tile(reshaped_subarray_2, [1, 1, 1, 1, level*level*(level-1)])

reshape_tiled_subarray_1 = tf.reshape(tiled_subarray_1, shape=(batch, sub_row, sub_column, channel, level, level, level-1))
reshape_tiled_subarray_2 = tf.reshape(tiled_subarray_2, shape=(batch, sub_row, sub_column, channel, level, level, level-1))

print("reshape_tiled_subarray_1: ", reshape_tiled_subarray_1)
print("reshape_tiled_subarray_2", reshape_tiled_subarray_2)



reshaped_subarray_1:  Tensor("Reshape_2:0", shape=(1, 3, 1, 1, 1), dtype=int32)
reshaped_subarray_2:  Tensor("Reshape_3:0", shape=(1, 3, 1, 1, 1), dtype=int32)
reshape_tiled_subarray_1:  Tensor("Reshape_4:0", shape=(1, 3, 1, 1, 3, 3, 2), dtype=int32)
reshape_tiled_subarray_2 Tensor("Reshape_5:0", shape=(1, 3, 1, 1, 3, 3, 2), dtype=int32)


In [7]:
# Now, to calculate the GLCM, we need a sequence constant in (0, ... , level-1).
# To applied matrix operation on it, we need to initial it in Tensor type.

# For it is so complicated to initial in pure tensorflow functions, we use numpy array first.
# Important: it is OK to use some python or numpy functions in tensorflow,
# but can't be done with functions that depends on back propogation.

#define the shape of sequence, which is something like: 
# [0, 1, 2, ... , level-1] 
# [0, 1, 2, ... , level-1]          
# [0, 1, 2, ... , level-1]
#           .
#           .
#           .
# and delete i at ith row. 

#raw sequence
sequence = tf.constant([])

for i in range(level):
    hold = tf.ones(shape=(level-1))*i
    sequence = tf.concat([sequence, hold], axis=0)
    
sequence = tf.cast(sequence, tf.int32)
    
#print("raw_sequence: ")
#print(K.eval(sequence))
    
sequence_reshape = tf.reshape(sequence, (level-1, level))
sequence_transpose = tf.transpose(sequence_reshape)
sequence_rev = tf.reverse(sequence_transpose, [0])

print("sequence_rev: ")
print(K.eval(sequence_rev))

print(sequence_rev)




sequence_rev: 
[[1 2]
 [0 2]
 [0 1]]
Tensor("ReverseV2:0", shape=(3, 2), dtype=int32)


In [8]:
# To show the arrays of each step, we show it in a single entry case.
# It will take a longer time to run while taking a single ertry from raw image, 
# please wait patiently.

#tile it(Demo version)
sequence_rev_reshape = tf.reshape(sequence_rev, (1, level, level-1))
sequence_rev_reshape_tile = tf.tile(sequence_rev_reshape, [level, 1, 1])

print("sequence_rev_reshape")
print(K.eval(sequence_rev_reshape_tile))
print("\n")

print("sequence_rev_reshape_tile")
print(sequence_rev_reshape_tile.shape)
print("\n")

#GLCM_Denominator(Demo version)
sequence_Denominator_reshape = tf.reshape(sequence, (level, level-1))
GLCM_t = tf.tile(tf.reshape(sequence_Denominator_reshape, (1, level, level-1)), [level, 1, 1])
print("GLCM_t: ")
print(K.eval(GLCM_t))
print("\n")

GLCM_Denominator_array = GLCM_t-sequence_rev_reshape_tile
GLCM_Denominator = tf.reduce_prod(GLCM_Denominator_array, axis=2)



print("GLCM_Denominator_array[0]: ")
print(K.eval(GLCM_Denominator_array[0]))
print("\n")

print("GLCM_Denominator: ")
print(K.eval(GLCM_Denominator))
print("\n")

#GLCM_Numerator(Demo version)
GLCM_Numerator_1_array = reshape_tiled_subarray_1[0][0][0][0]-sequence_rev_reshape_tile
GLCM_Numerator_2_array = reshape_tiled_subarray_2[0][0][0][0]-sequence_rev_reshape_tile

GLCM_Numerator_1 = tf.reduce_prod(GLCM_Numerator_1_array, axis=2)
GLCM_Numerator_2 = tf.reduce_prod(GLCM_Numerator_2_array, axis=2)

print("reshape_tiled_subarray_1[0][0][0][0]: ")
print(K.eval(reshape_tiled_subarray_1[0][0][0][0]))

print("GLCM_Numerator_1_array[0]: ")
print(K.eval(GLCM_Numerator_1_array[0]))

print("GLCM_Numerator_1: ")
print(K.eval(GLCM_Numerator_1))
print("\n")

#print("GLCM_Numerator_1")
#print(K.eval(GLCM_Numerator_1))

# After all, the result of this step is not GLCM yet, 
# it still need to be done with some matrix operation.
GLCM_Single_Entry = GLCM_Numerator_1/GLCM_Denominator
print("GLCM_Single_Entry: ")
print(K.eval(GLCM_Single_Entry))





sequence_rev_reshape
[[[1 2]
  [0 2]
  [0 1]]

 [[1 2]
  [0 2]
  [0 1]]

 [[1 2]
  [0 2]
  [0 1]]]


sequence_rev_reshape_tile
(3, 3, 2)


GLCM_t: 
[[[0 0]
  [1 1]
  [2 2]]

 [[0 0]
  [1 1]
  [2 2]]

 [[0 0]
  [1 1]
  [2 2]]]


GLCM_Denominator_array[0]: 
[[-1 -2]
 [ 1 -1]
 [ 2  1]]


GLCM_Denominator: 
[[ 2 -1  2]
 [ 2 -1  2]
 [ 2 -1  2]]


reshape_tiled_subarray_1[0][0][0][0]: 
[[[2 2]
  [2 2]
  [2 2]]

 [[2 2]
  [2 2]
  [2 2]]

 [[2 2]
  [2 2]
  [2 2]]]
GLCM_Numerator_1_array[0]: 
[[1 0]
 [2 0]
 [2 1]]
GLCM_Numerator_1: 
[[0 0 2]
 [0 0 2]
 [0 0 2]]


GLCM_Single_Entry: 
[[ 0. -0.  1.]
 [ 0. -0.  1.]
 [ 0. -0.  1.]]


In [9]:
#tile it 
# Flatten it for reason of too much dimension.
sequence_rev_reshape = tf.reshape(sequence_rev, (1, 1, 1, 1, 1, level*(level-1)))
sequence_rev_reshape_tile = tf.tile(sequence_rev_reshape, [batch, sub_row, sub_column, channel, level, 1])
sequence_rev_reshaped_tile = tf.reshape(sequence_rev_reshape_tile, (batch, sub_row, sub_column, channel, level, level, level-1))

print("sequence_rev_reshape")
print(K.eval(sequence_rev_reshape))

print(sequence_rev_reshape_tile.shape)

sequence_rev_reshape
[[[[[[1 2 0 2 0 1]]]]]]
(1, 3, 1, 1, 3, 6)


In [10]:
#Lagrange polynomials Denominator
GLCM_t = tf.tile(tf.reshape(sequence, (1, 1, 1, 1, level*(level-1))), [batch, sub_row, sub_column, channel, level])
GLCM_t_reshape = tf.reshape(GLCM_t, (batch, sub_row, sub_column, channel, level, level, (level-1)))
GLCM_Denominator_array = GLCM_t_reshape-sequence_rev_reshaped_tile
GLCM_Denominator = tf.reduce_prod(GLCM_Denominator_array, axis=6)

print("GLCM_Denominator.shape: ")
print(GLCM_Denominator.shape)

#print("GLCM_Denominator[0][0][0]")
#print(K.eval(GLCM_Denominator[0][0][0]))

#Lagrange polynomials Numerator
GLCM_Numerator_1_array = reshape_tiled_subarray_1-sequence_rev_reshaped_tile
GLCM_Numerator_2_array = reshape_tiled_subarray_2-sequence_rev_reshaped_tile

GLCM_Numerator_1 = tf.reduce_prod(GLCM_Numerator_1_array, axis=6)
GLCM_Numerator_2 = tf.reduce_prod(GLCM_Numerator_2_array, axis=6)

print("GLCM_Numerator_1.shape: ")
print(GLCM_Numerator_1.shape)
print("\n")

GLCM_subarray_1 = GLCM_Numerator_1/GLCM_Denominator
GLCM_subarray_2 = GLCM_Numerator_2/GLCM_Denominator
print("GLCM_subarray: ")
print(GLCM_subarray_1.shape)

print("GLCM_subarray_2")
print(K.eval(GLCM_subarray_2 ))



GLCM_Denominator.shape: 
(1, 3, 1, 1, 3, 3)
GLCM_Numerator_1.shape: 
(1, 3, 1, 1, 3, 3)


GLCM_subarray: 
(1, 3, 1, 1, 3, 3)
GLCM_subarray_2
[[[[[[ 0. -0.  1.]
     [ 0. -0.  1.]
     [ 0. -0.  1.]]]]



  [[[[ 1. -0.  0.]
     [ 1. -0.  0.]
     [ 1. -0.  0.]]]]



  [[[[ 0. -0.  1.]
     [ 0. -0.  1.]
     [ 0. -0.  1.]]]]]]


In [11]:
# Now, we need to do logic checking and merging on our subarrays to get the final GLCM.

# a and b = a*b
# We have to do transpose on GLCM_subarray_2 for its different location with GLCM_subarray_1 in GLCM. 

GLCM_subarray_2_transpose = tf.transpose(GLCM_subarray_2, perm=[0, 1, 2, 3, 5, 4])
GLCM_single_entry = tf.multiply(GLCM_subarray_1, GLCM_subarray_2_transpose)
print("GLCM_single_entry1: ")
print(K.eval(GLCM_subarray_1))
print("GLCM_single_entry2_transpose: ")
print(K.eval(GLCM_subarray_2_transpose))

# sum single entries to get the final GLCM.
GLCM = tf.reduce_sum(GLCM_single_entry, axis=[1, 2])

if symmetric:
    GLCM = GLCM+tf.transpose(GLCM, perm=[0, 1, 3, 2])
if normed:
    GLCM = GLCM/tf.reduce_sum(GLCM)
    

GLCM_single_entry1: 
[[[[[[ 0. -0.  1.]
     [ 0. -0.  1.]
     [ 0. -0.  1.]]]]



  [[[[ 0. -0.  1.]
     [ 0. -0.  1.]
     [ 0. -0.  1.]]]]



  [[[[ 0. -0.  1.]
     [ 0. -0.  1.]
     [ 0. -0.  1.]]]]]]
GLCM_single_entry2_transpose: 
[[[[[[ 0.  0.  0.]
     [-0. -0. -0.]
     [ 1.  1.  1.]]]]



  [[[[ 1.  1.  1.]
     [-0. -0. -0.]
     [ 0.  0.  0.]]]]



  [[[[ 0.  0.  0.]
     [-0. -0. -0.]
     [ 1.  1.  1.]]]]]]


In [12]:
print(K.eval(GLCM))

[[[[ 0.          0.          0.16666667]
   [ 0.          0.          0.        ]
   [ 0.16666667  0.          0.66666667]]]]


In [17]:
print(GLCM.dtype)

<dtype: 'float64'>


In [14]:
x = tf.constant([[[ 1,  2,  3],
                  [ 4,  5,  6]],
                 [[ 7,  8,  9],
                  [10, 11, 12]]])

In [15]:
x.shape

TensorShape([Dimension(2), Dimension(2), Dimension(3)])

In [16]:
tf.transpose(x, perm=[0, 2, 1]).shape

TensorShape([Dimension(2), Dimension(3), Dimension(2)])